# Instructions

1.  Download this dataset:  https://www.kaggle.com/slothkong/10-monkey-species  
        You may need to create a kaggle account / sign in to accomplish this.
        
2. Git clone this repo or download this notebook.
3. Create a new folder, data in the same directory this notebook is located
4. Unzip the monkey dataset in this folder.  Then unzip the train.zip and validation.zip folders.
5. The directory hierarchy should be ../data/10-monkey-species/training and ../data/10-monkey-species/validation.  Each of these subdirectories should contain a directory per label, n0 thru n9.

6. Implement the code in build_model()

** You will most certainly need access to a GPU for this assignment **

## Questions

In addition to implementing build_model(), answer these questions.

1.  Is this network a high bias or high variance model?  Refer to chapter 1 and look at this model in tensorboard if you're unsure.

2. What is the likely cause of the issue you identified in the previous question?

3.  What might you do to make this network perform better?

In [ ]:
# these seeds are both required for reproducibility
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dropout, Input
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import os

In [ ]:
class Configuration:
    def __init__(self):
        self.feature_extraction_epochs = 10
        self.fine_tuning_epochs = 20
        self.epochs_without_transfer_learning = 100
        self.batch_size = 30
        self.data_dir = "data/10-monkey-species/training"
        self.val_dir = "data/10-monkey-species/validation"

In [ ]:
def build_model():
    inputs = Input(shape=(299,299,3), name="input")
    
    """YOUR CODE HERE
    Network Architecture should be
   
    Conv2D 128 units, 3x3 kernel, relu activation
    MaxPooling2D 2x2 pool size
    
    Conv2D 64 units, 3x3 kernel, relu activation
    MaxPooling2D 2x2 pool size
    
    Conv2D 32 units, 3x3 kernel, relu activation
    MaxPooling2D 2x2 pool size
    
    Conv2D 16 units, 3x3 kernel, relu activation
    MaxPooling2D 2x2 pool size
    
    Dense 1024
    Dense softmax layer (named "output")
    
    
    """

    # finalize and compile
    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
def create_callbacks(name):
    tensorboard_callback = TensorBoard(log_dir=os.path.join(os.getcwd(), "tensorboard_log", name), write_graph=True, write_grads=False)
    checkpoint_callback = ModelCheckpoint(filepath="./model-weights-" + name + ".{epoch:02d}-{val_loss:.6f}.hdf5", monitor='val_loss',
                                          verbose=0, save_best_only=True)
    return [tensorboard_callback]

In [ ]:
def setup_data(train_data_dir, val_data_dir, img_width=299, img_height=299, batch_size=16):
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')
    return train_generator, validation_generator

In [ ]:
def fit_model(model, train_generator, val_generator, batch_size, epochs, name):
    model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_generator.n // batch_size,
        callbacks=create_callbacks(name=name),
        verbose=1)
    return model

In [ ]:
def eval_model(model, val_generator, batch_size):
    scores = model.evaluate_generator(val_generator, steps=val_generator.n // batch_size)
    print("Loss: " + str(scores[0]) + " Accuracy: " + str(scores[1]))

## Load configuration class
This has all our model configuration parameters in it.  It's defined above.

In [ ]:
config = Configuration()

## Load Data
If everything is setup correctly, this method will print  
"  
Found 1097 images belonging to 10 classes.  
Found 272 images belonging to 10 classes.  
"  



In [ ]:
train_generator, val_generator = setup_data(config.data_dir, config.val_dir, batch_size=config.batch_size)

## Compile your model and print a summary

Your network should look like this.


input (InputLayer)           (None, 299, 299, 3)       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 297, 297, 128)     3584      
_________________________________________________________________
pool_1 (MaxPooling2D)        (None, 148, 148, 128)     0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 146, 146, 64)      73792     
_________________________________________________________________
pool_2 (MaxPooling2D)        (None, 73, 73, 64)        0         
_________________________________________________________________
conv_3 (Conv2D)              (None, 71, 71, 32)        18464     
_________________________________________________________________
pool_3 (MaxPooling2D)        (None, 35, 35, 32)        0         
_________________________________________________________________
conv_4 (Conv2D)              (None, 33, 33, 16)        4624      
_________________________________________________________________
pool_4 (MaxPooling2D)        (None, 16, 16, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              4195328   
_________________________________________________________________
softmax (Dense)              (None, 10)                10250     

Total params: 4,306,042
Trainable params: 4,306,042
Non-trainable params: 0
_________________________________________________________________

In [ ]:
model = build_model()
print (model.summary())

## Model Training

You're most certainly going to want a GPU to train this model.


In [ ]:
model = fit_model(model, train_generator, val_generator,
                  batch_size=config.batch_size,
                  epochs=config.epochs_without_transfer_learning,
                  name="without_transfer_learning")

In [ ]:
# Evaluate your model.
eval_model(model, val_generator, batch_size=config.batch_size)

In [ ]:
# Save your model weights
model.save("no_transfer_learning_model.h5")